In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import re, os, csv
import pickle
file = 'combined_sm.csv'

In [2]:
def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

def split(data, name, s, label_encoder_path='data_file/enc.pickle', ids=None, calc_minval=True):
    ensure_dir(name)
    np.random.seed(s)
    with open(label_encoder_path, 'rb') as handle:
        label_encoder = pickle.load(handle)
    data['encseq'] = data['Modified_sequence'].apply(lambda x: label_encoder.transform(list(x)))
    if calc_minval:
        data['minval'] = np.min(data['label'])
        data['maxval'] = np.max(data['label'])
    else:
        data['minval']=275.440277
        data['maxval']=1112.030762
        
    if ids == None:
        #a = np.random.uniform(0.0,1.0,len(data)) > 0.98
        data['test'] = data['PT']
        print('Using proteome tools testsset')
        
    else:
        print('using predefined testset')
        data['test'] = ~data['Modified_sequence'].isin(ids)

    data['task'] = 0
    print('Name: ', name, 'Seed: ', s, 'Len test: ', len(data[data['test']]),'Len set test: ', len(set(data[data['test']])),'Len not test: ', len(data[~data['test']]),'Len set not test: ', len(set(data[~data['test']])))
    data[~data['test']].to_pickle(name + str(s) + '_train.pkl')
    data[data['test']].to_pickle(name +str(s) + '_test.pkl')
    return data

In [3]:
data = pd.read_csv(file)
data = data.drop('Unnamed: 0', axis = 1)
# print(data.head())
data = data.rename(index = str, columns = {"Modified sequence": "Modified_sequence"})
data["Modified_sequence"] = data["Modified_sequence"].str.replace('_', '')
data.head()

,Modified_sequence,Charge,Mass,Intensity,Retention time,CCS,PT
0,(ac)AAAAAAAAAAGAAGGR,2,1239.63200,149810.0,70.140,409.092529,False
1,(ac)AAAAAAAAEQQSSNGPVKK,2,1810.91734,21349.0,19.645,481.229248,True
2,(ac)AAAAAAAGAAGSAAPAAAAGAPGSGGAPSGSQGVLIGDR,3,3144.55482,194000.0,3947.700,772.098084,False
3,(ac)AAAAAAAGDSDSWDADAFSVEDPVRK,2,2634.18340,6416400.0,94.079,573.213196,False
4,(ac)AAAAAAAGDSDSWDADAFSVEDPVRK,3,2634.18340,5400600.0,94.841,635.000549,False


In [4]:
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
dat = data["Modified_sequence"]
dat = [list(d) for d in dat]

flat_list = ['_'] + [item for sublist in dat for item in sublist]
values = np.array(flat_list)

label_encoder = LabelEncoder()
label_encoder.fit(values)
ensure_dir('data_file/enc.pickle')
ensure_dir('data_file/enc_list.csv')
with open('data_file/enc.pickle', 'wb') as handle:
    pickle.dump(label_encoder, handle)
import csv
with open('data_file/enc_list.csv', 'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(list(label_encoder.classes_))

In [5]:
print(label_encoder.classes_, len(label_encoder.classes_))
# ['_']在编码器中的数值表示
print(label_encoder.transform([['_']]), 22)

['(' ')' 'A' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'K' 'L' 'M' 'N' 'P' 'Q' 'R' 'S'
 'T' 'V' 'W' 'Y' '_' 'a' 'c' 'o' 'x'] 27
[22] 22


E:\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [6]:
data.shape

(718917, 7)

In [8]:
data = data[~data['Intensity'].isnull()]
data = data[~data['CCS'].isnull()]
d = data
d['label'] = d['CCS'].values.tolist()
outpath = './'
d = split(d, outpath, 2)

Using proteome tools testsset
Name:  ./ Seed:  2 Len test:  185248 Len set test:  13 Len not test:  533669 Len set not test:  13


In [ ]:
d.head()
d.to_csv('data_final.csv')